# Autoencoder

## Prepare Data

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')

training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t')
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t')
training_set.head()

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [2]:
training_set = np.array(training_set, dtype='int')
test_set = np.array(test_set, dtype='int')

# get the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
print(nb_users)
print(nb_movies)

943
1682


In [3]:
# converting the data into a matrix with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_user in range(1, nb_users+1):
        id_movies = data[:,1][data[:,0] == id_user]
        id_ratings = data[:,2][data[:,0] == id_user]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data
            
training_set = convert(training_set)
test_set = convert(test_set)    

# converting data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Build the Autoencoder

In [32]:
class StackedAutoEncoder(nn.Module):

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
        
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [33]:
auto_encoder = StackedAutoEncoder()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(auto_encoder.parameters(), lr=0.01, weight_decay=0.5)

## Training Autoencoder

In [35]:
nb_epochs = 200
for epoch in range(1, nb_epochs+1):
    train_loss = 0
    users_rated_count = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        #test that the user rated at least one movie (optimize memory)
        if torch.sum(target.data > 0) > 0:  
            output = auto_encoder(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            users_rated_count += 1.
            optimizer.step()
    print(f'epoch: {epoch} loss: {train_loss/users_rated_count}')

epoch: 1 loss: 1.7658158540725708
epoch: 2 loss: 1.0965160131454468
epoch: 3 loss: 1.0534982681274414
epoch: 4 loss: 1.0381323099136353
epoch: 5 loss: 1.030817985534668
epoch: 6 loss: 1.0267314910888672
epoch: 7 loss: 1.023602843284607
epoch: 8 loss: 1.0220305919647217
epoch: 9 loss: 1.0207315683364868
epoch: 10 loss: 1.0194804668426514
epoch: 11 loss: 1.0189080238342285
epoch: 12 loss: 1.0183860063552856
epoch: 13 loss: 1.0178974866867065
epoch: 14 loss: 1.0175381898880005
epoch: 15 loss: 1.0171769857406616
epoch: 16 loss: 1.0170212984085083
epoch: 17 loss: 1.0167031288146973
epoch: 18 loss: 1.0165205001831055
epoch: 19 loss: 1.0162731409072876
epoch: 20 loss: 1.016390323638916
epoch: 21 loss: 1.0160131454467773
epoch: 22 loss: 1.0159878730773926
epoch: 23 loss: 1.0157746076583862
epoch: 24 loss: 1.0156831741333008
epoch: 25 loss: 1.0157876014709473
epoch: 26 loss: 1.0157089233398438
epoch: 27 loss: 1.0152873992919922
epoch: 28 loss: 1.015108346939087
epoch: 29 loss: 1.012308955192566

##  Testing Autoencoder

In [39]:
test_loss = 0
users_rated_count = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    #test that the user rated at least one movie (optimize memory)
    if torch.sum(target.data > 0) > 0:  
        output = auto_encoder(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        users_rated_count += 1.
print(f'test loss: {test_loss/users_rated_count}')

test loss: 0.9507325291633606
